## Trabalho

In [1]:
from qgis.core import *
import sys
import os
from qgis.PyQt.QtCore import QVariant
from qgis.PyQt.QtGui import QColor, QImage
from qgis.PyQt.QtCore import QSize, QBuffer, QIODevice
import pandas as pd

In [2]:
QgsApplication.setPrefixPath(r"C:\OSGeo4W64\apps\qgis", True)

qgs = QgsApplication([], True)

qgs.initQgis()

sys.path.append(r'C:\OSGeo4W64\apps\qgis\python\plugins')

### Carregar informação acerca do Covid-19

In [3]:
path_to_csv = "file:///C:/Users/pmcca/Documents/04-26-2020.csv?delimiter=,"

csv_layer = QgsVectorLayer(path_to_csv, "EPID", "delimitedtext")
if not csv_layer.isValid():
    print("Layer failed to load!")
else:
    print('Load with sucess')

QgsProject.instance().addMapLayer(csv_layer)

Load with sucess


<QgsMapLayer: 'EPID' (delimitedtext)>

### Carregar csv com população do USA por estado

In [4]:
path_to_pop = "file:///C:/Users/pmcca/Documents/SCPRC-EST2019-18POP-RES.csv?delimiter=,"

pop_layer = QgsVectorLayer(path_to_pop, "POP", "delimitedtext")
if not pop_layer.isValid():
    print("Layer failed to load!")
else:
    print('Load with sucess')

QgsProject.instance().addMapLayer(pop_layer)

Load with sucess


<QgsMapLayer: 'POP' (delimitedtext)>

### Carregar informação acerca do shp com informação acerca dos USA

In [5]:
path_to_shp = r"data\usa_states_map\tl_2017_us_state.shp"

shp_layer = QgsVectorLayer(path_to_shp, "US", "ogr")
if not shp_layer.isValid():
    print("Layer failed to load!")
else:
    print('Load with sucess')
    
QgsProject.instance().addMapLayer(shp_layer)

provider = shp_layer.dataProvider()

Load with sucess


### Verificar os fields do shp

In [6]:
for feature in shp_layer.getFeatures():
    print(feature['NAME'])

West Virginia
Florida
Illinois
Minnesota
Maryland
Rhode Island
Idaho
New Hampshire
North Carolina
Vermont
Connecticut
Delaware
New Mexico
California
New Jersey
Wisconsin
Oregon
Nebraska
Pennsylvania
Washington
Louisiana
Georgia
Alabama
Utah
Ohio
Texas
Colorado
South Carolina
Oklahoma
Tennessee
Wyoming
Hawaii
North Dakota
Kentucky
United States Virgin Islands
Commonwealth of the Northern Mariana Islands
Guam
Maine
New York
Nevada
Alaska
American Samoa
Michigan
Arkansas
Mississippi
Missouri
Montana
Kansas
Indiana
Puerto Rico
South Dakota
Massachusetts
Virginia
District of Columbia
Iowa
Arizona


In [7]:
#for field in shp_layer.fields():
#    print(field.name())

### Join do COvid com USA

In [8]:
shpField='NAME'
csvField='Province_State'
joinObject = QgsVectorLayerJoinInfo()
joinObject.setJoinFieldName(csvField)
joinObject.setTargetFieldName(shpField)
joinObject.setJoinLayerId(csv_layer.id())
joinObject.setUsingMemoryCache(True)
joinObject.setJoinLayer(csv_layer)
shp_layer.addJoin(joinObject)

True

### Join do POP com USA

In [9]:
shpField='NAME'
csvField='NAME'
joinObject = QgsVectorLayerJoinInfo()
joinObject.setJoinFieldName(csvField)
joinObject.setTargetFieldName(shpField)
joinObject.setJoinLayerId(pop_layer.id())
joinObject.setUsingMemoryCache(True)
joinObject.setJoinLayer(pop_layer)
shp_layer.addJoin(joinObject)

True

In [10]:
caps = shp_layer.dataProvider().capabilities()

#print(shp_layer.dataProvider().capabilitiesString())

In [11]:
for field in shp_layer.fields():
    print(field.name())

REGION
DIVISION
STATEFP
STATENS
GEOID
STUSPS
NAME
LSAD
MTFCC
FUNCSTAT
ALAND
AWATER
INTPTLAT
INTPTLON
EPID_Country_Region
EPID_Last_Update
EPID_Lat
EPID_Long_
EPID_Confirmed
EPID_Deaths
EPID_Recovered
EPID_Active
EPID_FIPS
EPID_Incident_Rate
EPID_People_Tested
EPID_People_Hospitalized
EPID_Mortality_Rate
EPID_UID
EPID_ISO3
EPID_Testing_Rate
EPID_Hospitalization_Rate
POP_SUMLEV
POP_REGION
POP_DIVISION
POP_STATE
POP_POPESTIMATE2019
POP_POPEST18PLUS2019
POP_PCNT_POPEST18PLUS


In [12]:
for feature in shp_layer.getFeatures():
    print(feature['NAME'], feature['POP_POPESTIMATE2019'], feature['EPID_Confirmed'])

West Virginia 1792147 1055
Florida 21477737 31532
Illinois 12671821 43903
Minnesota 5639632 3602
Maryland 6045680 18581
Rhode Island 1059361 7439
Idaho 1787065 1897
New Hampshire 1359711 1864
North Carolina 10488084 8997
Vermont 623989 851
Connecticut 3565287 25269
Delaware 973764 4034
New Mexico 2096829 2726
California 39512223 43558
New Jersey 8882190 109038
Wisconsin 5822434 5912
Oregon 4217737 2311
Nebraska 1934408 3028
Pennsylvania 12801989 42616
Washington 7614893 13521
Louisiana 4648794 26773
Georgia 10617423 23486
Alabama 4903185 6421
Utah 3205958 4123
Ohio 11689100 15972
Texas 28995881 24967
Colorado 5758736 13441
South Carolina 5148714 5498
Oklahoma 3956971 3254
Tennessee 6829174 9667
Wyoming 578759 502
Hawaii 1415872 606
North Dakota 762062 867
Kentucky 4467673 4085
United States Virgin Islands None None
Commonwealth of the Northern Mariana Islands None None
Guam None 141
Maine 1344212 1015
New York 19453561 288045
Nevada 3080156 4734
Alaska 731545 340
American Samoa None 0


### Colocar os valores do Covid a 0 para o caso de ser None

In [13]:
updateMap = {}
id = provider.fields().indexFromName( 'EPID_Confirmed' )
idd = provider.fields().indexFromName( 'EPID_Deaths' )

for feature in shp_layer.getFeatures():

    if feature['EPID_Confirmed'] is None:
        #Juntar o if para verificar se é possivel
        updateMap[feature.id()] = { id: 0, idd: 0 }

provider.changeAttributeValues( updateMap )

True

### Eliminar aqueles q n tem dados da população

In [14]:
population = []
cases = []
deaths = []
updateMap = {}
delete = []

for feature in shp_layer.getFeatures():

    if feature['POP_POPESTIMATE2019'] is None:#Mudar o EPID_Confirmed
        delete.append(feature.id())
    else:
        population.append(feature['POP_POPESTIMATE2019'])
        cases.append(feature['EPID_Confirmed'])
        deaths.append(feature['EPID_Deaths'])


res = shp_layer.dataProvider().deleteFeatures(delete)
population.sort()
cases.sort()
deaths.sort()

In [15]:
#for feature in shp_layer.getFeatures():
#    print(feature['NAME'], feature['EPID_Confirmed'])

Adicionar a coluna com %Infected, %Population e %Dead of the USA

In [16]:
def already_added():
    for field in shp_layer.fields():
        if field.name() == 'Inf(%USA)':
            return False
    return True

if caps & QgsVectorDataProvider.AddAttributes:
    print('Add field')
    res = shp_layer.dataProvider().addAttributes(
        [QgsField('Inf(%USA)', QVariant.Double), QgsField('Pop(%USA)', QVariant.Double), 
        QgsField('Dead(%USA)', QVariant.Double)])
    shp_layer.updateFields()

Add field


Adicionar valores ao %Infected, 

In [17]:
updateMap = {}
perc_infected = provider.fields().indexFromName( 'Inf(%USA)' )
perc_pop = provider.fields().indexFromName( 'Pop(%USA)' )
perc_dead = provider.fields().indexFromName( 'Dead(%USA)' )

values_inf = sum(cases)
values_pop = sum(population)
values_dead = sum(deaths)

print(values_inf, values_pop, values_dead)

for feature in shp_layer.getFeatures():
    i = (feature['EPID_Confirmed'] / values_inf) * 100
    p = (feature['POP_POPESTIMATE2019'] / values_pop) * 100
    d = (feature['EPID_Deaths'] / values_dead) * 100
    updateMap[feature.id()] = { perc_infected: i , perc_pop: p, perc_dead: d}
provider.changeAttributeValues( updateMap )

964048 328239523 54783


True

In [18]:
#for field in shp_layer.fields():
#    print(field.name())

In [19]:
#for feature in shp_layer.getFeatures():
#    print(feature['Name'], feature['Inf(%USA)'], feature['Pop(%USA)'], feature['Dead(%USA)'])

Metodo que cria um rendered para imprimir mapas

In [20]:
def apply_graduated_symbology(target):
    target_field = target
    myRangeList = []
    
    values_list = []
    
    for feature in shp_layer.getFeatures():
        values_list.append(feature[target])
    values_list.sort()
     
    miin = min(values_list)
    maax = max(values_list)
    
    symbol = QgsSymbol.defaultSymbol(shp_layer.geometryType())     
    symbol.setColor(QColor(255, 20, 20))                              
    myRange = QgsRendererRange(0, values_list[0], symbol, 'Group 0')                   
    myRangeList.append(myRange)                                     

    i = 0
    for value in values_list:
        if i > 0:
            symbol = QgsSymbol.defaultSymbol(shp_layer.geometryType())
            red = ((values_list[i] - miin) / (maax - miin)) * 255
            symbol.setColor(QColor(255 - red, 20, 20))
            myRange = QgsRendererRange(values_list[i - 1], value, symbol, 'Group ' + str(i))
            myRangeList.append(myRange)
            i = i + 1
        else:
            i = i + 1

    myRenderer = QgsGraduatedSymbolRenderer(target_field, myRangeList)  
    myRenderer.setMode(QgsGraduatedSymbolRenderer.Custom)               

    shp_layer.setRenderer(myRenderer)                                  
    
    print(f"Graduated color scheme applied")

In [21]:
country = QgsProject.instance().mapLayersByName('US')[0]

In [22]:
def getImage():
    xt = country.extent()
    # print(xt)
    width  = 2000
    height = int(width*xt.height()/xt.width())
    print("Gerar mapa com {} por {}".format(width, height))

    options = QgsMapSettings()
    options.setLayers([country])
    options.setBackgroundColor(QColor(255, 255, 255))
    options.setOutputSize(QSize(width, height))
    options.setExtent(xt)

    render = QgsMapRendererParallelJob(options)
    render.start()
    render.waitForFinished()

    image = render.renderedImage()
    return image

In [23]:
from IPython.display import Image

In [24]:
apply_graduated_symbology('EPID_Confirmed')

imgbuf= QBuffer()
imgbuf.open( QIODevice.ReadWrite )
image = getImage()
image.save( imgbuf,"PNG" )
Image( imgbuf.data() )

C:\OSGEO4~1\apps\Python37\lib\site-packages\ipykernel_launcher.py:32: DeprecationWarning: QgsGraduatedSymbolRenderer.setMode() is deprecated


Graduated color scheme applied
Gerar mapa com 20000 por 2928


In [25]:
apply_graduated_symbology('Pop(%USA)')

imgbuf= QBuffer()
imgbuf.open( QIODevice.ReadWrite )
image = getImage()
image.save( imgbuf,"PNG" )
Image( imgbuf.data() )

C:\OSGEO4~1\apps\Python37\lib\site-packages\ipykernel_launcher.py:32: DeprecationWarning: QgsGraduatedSymbolRenderer.setMode() is deprecated


Graduated color scheme applied
Gerar mapa com 20000 por 2928


In [26]:
qgs.exitQgis()